In [1]:
import gc
import os
from pathlib import Path
#import random
import sys

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import scipy as sp

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.core.display import display, HTML

# --- plotly --- 
from plotly import tools, subplots
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import plotly.io as pio
pio.templates.default = "plotly_dark"

# --- dash ---
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output




In [2]:
from datetime import datetime

def _convert_date_str(df):
    try:
        df.columns = list(df.columns[:1]) + [datetime.strptime(d, "%m%d%y").date().strftime("%d/%m/%Y") for d in df.columns[1:]]
    except:
        print('_convert_date_str failed with %y, try %Y')
        df.columns = list(df.columns[:1]) + [datetime.strptime(d, "%m%d%Y").date().strftime("%d/%m/%Y") for d in df.columns[1:]]


In [3]:
cases_df=pd.read_csv('dataset.csv')
cases_df.head(5)

,CNTRYNAME,122,123,124,125,126,127,128,129,130,...,416,417,418,419,420,421,422,423,424,425
0,Aruba,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Antigua and Barbuda,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,23.0,23.0,23.0,23.0,23.0,23.0,24.0,24.0,24.0,24.0
2,Afghanistan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,840.0,906.0,933.0,996.0,1026.0,1092.0,1176.0,1279.0,1351.0,1463.0
3,Algeria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2268.0,2418.0,2534.0,2629.0,2718.0,2811.0,2910.0,3007.0,3127.0,3256.0
4,Azerbaijan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1283.0,1340.0,1373.0,1398.0,1436.0,1480.0,1518.0,1548.0,1592.0,1617.0


In [4]:
#renaming the column labels

#df = df.rename(columns=lambda x: x+'x') 
cases_df= cases_df.rename(columns=lambda x:'0'+x+'2020')
cases_df= cases_df.rename(columns={"0CNTRYNAME2020":"Country"})
_convert_date_str(cases_df)
cases_df.tail(5)

_convert_date_str failed with %y, try %Y


,Country,22/01/2020,23/01/2020,24/01/2020,25/01/2020,26/01/2020,27/01/2020,28/01/2020,29/01/2020,30/01/2020,...,16/04/2020,17/04/2020,18/04/2020,19/04/2020,20/04/2020,21/04/2020,22/04/2020,23/04/2020,24/04/2020,25/04/2020
249,Zambia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,48.0,52.0,57.0,61.0,65.0,70.0,74.0,76.0,84.0,84.0
250,Zimbabwe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,23.0,24.0,25.0,25.0,25.0,28.0,28.0,28.0,29.0,31.0
251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
252,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
253,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
confirmed_melt_df=cases_df.melt(id_vars=['Country'], value_vars=cases_df.columns[1:],var_name='Date', value_name='Cases')

confirmed_melt_df['Date'] = pd.to_datetime(confirmed_melt_df['Date'], format='%d/%m/%Y')
confirmed_melt_df

,Country,Date,Cases
0,Aruba,2020-01-22,0.0
1,Antigua and Barbuda,2020-01-22,0.0
2,Afghanistan,2020-01-22,0.0
3,Algeria,2020-01-22,0.0
4,Azerbaijan,2020-01-22,0.0
...,...,...,...
24125,Zambia,2020-04-25,84.0
24126,Zimbabwe,2020-04-25,31.0
24127,NaN,2020-04-25,NaN
24128,NaN,2020-04-25,NaN


In [6]:
target_date = confirmed_melt_df['Date'].max()
global_df=confirmed_melt_df.query('Date == @target_date')
global_df['Confirmed'] = np.log10(global_df['Cases'] + 1)

In [7]:
covid_map = px.choropleth(global_df, locations="Country", 
            color="Confirmed",locationmode='country names',
            hover_data=['Cases'],
            range_color=[global_df['Confirmed'].min(), global_df['Confirmed'].max()], 
            color_continuous_scale="reds", 
            title='Countries with Confirmed Cases')
covid_map.show()


In [11]:
#Dash App Layout
news_api_key='1afe5c04580a4593a86c83c197c21a56'
app = dash.Dash(__name__)
app.layout = html.Div([
    
    html.H1("COVID-19 DASHBOARD",style={'text-align':'center'}),
    
    
    dcc.Graph(id='world_map',figure=covid_map),
    
    
    html.Br(),
    html.Div(id='stats_container', children=[])
    
])

In [ ]:

#Dash App Callback
@app.callback(
    [Output(component_id='stats_container', component_property='children'),
     Output(component_id='world_map', component_property='figure')]
)

def show_map():
    return covid_map


def dis_play_hover_data(hover_data):
    return json.dumps(hover_data, indent=2)

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)


In [10]:
import requests

API_KEY = '1afe5c04580a4593a86c83c197c21a56'

params = {
    'q': 'corona virus',
    'source': 'bbc-news',
    'sortBy': 'top',
    'language': 'en',
    #'category': 'business',
    #'country': 'us',
    #'apiKey': API_KEY,
}

headers = {
    'X-Api-Key': API_KEY,  # KEY in header to hide it from url
}

url = 'https://newsapi.org/v2/top-headlines'

response = requests.get(url, params=params, headers=headers)
data = response.json()

articles = data["articles"] 
results = [arr["title"] for arr in articles] 

for i, arr in enumerate(results, 1): 
    print(i, arr)

1 Coronavirus Briefing Newsletter - Times of India
2 Police crack down on secret warehouse parties violating Covid-19 guidelines - CNN Video
3 SA Chief Public Health Officer Nicola Spurrier apologises for wrongly suggesting man breached coronavirus quarantine
4 SA Chief Public Health Officer Nicola Spurrier apologises for wrongly suggesting man breached coronavirus quarantine
5 Americans brace for Covid 'surge upon a surge'
6 Covid infections in England fall by 30% over lockdown - React study
7 Coronavirus India live updates: TN extends lockdown till Dec 31 with several relaxations
8 Canada’s holiday coronavirus spread expected to eclipse first wave
9 Qantas to outsource more than 2,000 ground crew jobs due to coronavirus pandemic
10 “Sunday Morning” Full Episode 11/29
11 Coronavirus: Donnelly warns State entering ‘uncharted territory’
12 'A natural disaster ... in all 50 states' is unfolding just as travelers disperse nationwide after Thanksgiving
13 Top health officials agree America